In [31]:
import pandas as pd
from math import sqrt
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, BaggingRegressor
from google.colab import files

uploaded = files.upload()

Saving train_4.csv to train_4.csv


In [32]:
train = pd.read_csv("train_4.csv")
train.head(2)

,Unnamed: 0,id,carat,cut,color,clarity,x,y,z,price
0,0,0,0.30,3,6,1,4.31,4.28,2.68,6.353
1,1,1,1.01,4,5,5,6.42,6.46,4.04,9.183


In [33]:
train = train.drop(columns = ["Unnamed: 0"])

In [34]:
columnas = [i for i in train.columns if i not in ["price"]]
X = train[columnas]
y = train["price"]

Separamos y definimos variables respuesta y predictoras

In [35]:
train.head(2)

,id,carat,cut,color,clarity,x,y,z,price
0,0,0.30,3,6,1,4.31,4.28,2.68,6.353
1,1,1.01,4,5,5,6.42,6.46,4.04,9.183


In [36]:
import pandas as pd
import math
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [37]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)


Balanceamos el modelo

In [38]:
from sklearn.pipeline import Pipeline

In [39]:
pipe = Pipeline(steps=[("estimator", GradientBoostingRegressor())])

Inicializamos pipeline

In [40]:
params_grid = [{
                'estimator':[GradientBoostingRegressor()],
                 'estimator__n_estimators':[50,100,200,300],
                 'estimator__max_depth':[4, 5, 6, 7], 
                 'estimator__min_samples_split':[2,3], 
                 'estimator__min_samples_leaf':[1,2,3],
                }

              ]

Ejecutamos pipeline para evaluar cual es el mejor modelo

In [41]:
from sklearn.model_selection import GridSearchCV

In [42]:
grid = GridSearchCV(pipe, params_grid, n_jobs= -1, verbose = 3)

In [43]:
%%time
res = grid.fit(X, y)

Fitting 5 folds for each of 96 candidates, totalling 480 fits
CPU times: user 36 s, sys: 2.2 s, total: 38.2 s
Wall time: 59min 9s


Entrenamos modelos

In [44]:
best_model = grid.best_params_
best_model

{'estimator': GradientBoostingRegressor(max_depth=7, min_samples_leaf=3, n_estimators=200),
 'estimator__max_depth': 7,
 'estimator__min_samples_leaf': 3,
 'estimator__min_samples_split': 2,
 'estimator__n_estimators': 200}

Seleccionamos mejor modelo

In [45]:
results = pd.DataFrame(grid.cv_results_).head()
results.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_estimator,param_estimator__max_depth,param_estimator__min_samples_leaf,param_estimator__min_samples_split,param_estimator__n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,3.256288,0.023574,0.015986,0.000512,"GradientBoostingRegressor(max_depth=7, min_sam...",4,1,2,50,{'estimator': GradientBoostingRegressor(max_de...,0.987928,0.987823,0.987838,0.988286,0.987988,0.987972,0.000168,94
1,6.603175,0.024616,0.024249,0.000644,"GradientBoostingRegressor(max_depth=7, min_sam...",4,1,2,100,{'estimator': GradientBoostingRegressor(max_de...,0.990507,0.990407,0.990499,0.990702,0.990544,0.990532,0.000096,79
2,13.359357,0.063351,0.040210,0.004236,"GradientBoostingRegressor(max_depth=7, min_sam...",4,1,2,200,{'estimator': GradientBoostingRegressor(max_de...,0.991672,0.991744,0.991802,0.991989,0.991710,0.991783,0.000111,65
3,20.150244,0.060162,0.052042,0.001197,"GradientBoostingRegressor(max_depth=7, min_sam...",4,1,2,300,{'estimator': GradientBoostingRegressor(max_de...,0.991566,0.992112,0.992180,0.992315,0.991907,0.992016,0.000261,51
4,3.222453,0.013999,0.015178,0.000333,"GradientBoostingRegressor(max_depth=7, min_sam...",4,1,3,50,{'estimator': GradientBoostingRegressor(max_de...,0.987936,0.987823,0.987838,0.988286,0.987987,0.987974,0.000167,93


Mostramos resultados

In [46]:
from sklearn.model_selection import cross_validate

In [47]:
results = cross_validate(estimator=GradientBoostingRegressor(), 
               X=X, y=y, scoring='neg_root_mean_squared_error', cv=10, verbose = 3,  n_jobs= -1)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   28.8s finished


Validamos

In [48]:
rmse_ = abs(results["test_score"].mean())
rmse_

0.10127584582471469

Sacamos RMSE

In [49]:
gb = GradientBoostingRegressor()

In [50]:
gb.fit(X_train, y_train)

GradientBoostingRegressor()

fitteamos modelo

In [51]:
y_pred_train = gb.predict(X_train)

In [52]:
train_df = pd.DataFrame({'Real': y_train, 'Predicted': y_pred_train, 'Set': ['Train']*len(y_train)})
train_df.head(2)

,Real,Predicted,Set
30708,8.262,8.115513,Train
5625,6.981,6.978793,Train


In [53]:
from sklearn import metrics

In [54]:
def metricas(y_test, y_train, y_test_pred, y_train_pred, tipo_modelo):
    
    
    resultados = {'MAE': [metrics.mean_absolute_error(y_test, y_test_pred), metrics.mean_absolute_error(y_train, y_train_pred)],
                'MSE': [metrics.mean_squared_error(y_test, y_test_pred), metrics.mean_squared_error(y_train, y_train_pred)],
                'RMSE': [np.sqrt(metrics.mean_squared_error(y_test, y_test_pred)), np.sqrt(metrics.mean_squared_error(y_train, y_train_pred))],
                'R2':  [metrics.r2_score(y_test, y_test_pred), metrics.r2_score(y_train, y_train_pred)],
                 "set": ["test", "train"]}
    df = pd.DataFrame(resultados)
    df["modelo"] = tipo_modelo
    return df

Sacamos métricas del modelo

In [57]:
import pickle

In [60]:
def save_scaler(amount, filename = "scaler.pkl"):
    with open(filename, "wb") as f:
        pickle.dump(amount, f)
save_scaler(gb, filename="gb3.pkl")

Guardamos escalar para llevarnoslo a predecir

In [61]:
save_scaler(gb)

In [62]:
files.download('gb3.pkl')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Descargamos escalar